In [1]:
!nvidia-smi

Tue Nov 14 23:36:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install transformers==4.33.2
!pip3 install optimum==1.13.2
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
model_name_or_path = 'TheBloke/Llama-2-13B-German-Assistant-v4-GPTQ'  # "TheBloke/Llama-2-13B-chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map='auto',
                                             trust_remote_code=False,
                                             revision="gptq-4bit-32g-actorder_True")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

(…)/gptq-4bit-32g-actorder_True/config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/8.12G [00:00<?, ?B/s]

(…)32g-actorder_True/generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

(…)-GPTQ/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/568k [00:00<?, ?B/s]

(…)tant-v4-GPTQ/resolve/main/tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

(…)PTQ/resolve/main/special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [5]:
import pickle

with open("definitions.pickle", "rb") as file:
    definitions = pickle.load(file)

definitions

['Die "Negation:NEG_XgeschweigedennY-Konstruktion" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) "geschweige_denn", die internen Kern-Konstruktionselemente (Kern-KE) "Erstes_Konjunkt" und "Zweites_Konjunkt" sowie die externen Kern-KE "Negator" und "Fokuskontext". Das "Erste_Konjunkt" geht dem "KEE" voraus, auf welches da

In [6]:
import re
from tqdm import tqdm

def generate_examples(definition: str, temperature=0.75, top_p=0.95, top_k=1, max_new_tokens=1024):
    prompt = lambda definition: f'''Nenne fünf konkrete Beispiele für eine vordefinierte Satzkonstruktion. Der User definiert die Konstruktion. Alle Sätze müssen zur definierten Konstruktion passen. Gib nur die Sätze in einer Python-Liste aus. \n\n### User: Dies ist die Definition:\n{definition} \n\n### Assistant: Ausgabe: \n["'''
    #prompt = lambda definition: f'''[INST] <<SYS>>
    #Tell me five example sentences for a predefined German construction.
    #The definition will be provided in German.
    #Only output the example sentence.
    #Output the example sentences in a Python list.
    #Here are the construction definitions in German:
    #{definitions}
    #<</SYS>>
    #Output: ["'''

    prompt_length = len(prompt(definition))

    input_ids = tokenizer(prompt(definition), return_tensors='pt').input_ids.cuda()
    output = ""
    #while '.' not in output:
    output = model.generate(inputs=input_ids, temperature=0.75, do_sample=True, top_p=0.95, top_k=1, max_new_tokens=1024)
    output = tokenizer.decode(output[0])[prompt_length:].strip()
    #    print("\t" + output)
    return output

examples = []
for definition in tqdm(definitions):
    example = generate_examples(definition, temperature=0.95, max_new_tokens=300,
                                top_k=5)
    example = re.findall('\[\".*\"\]', example)
    try:
        print(example[0])
        examples.append(eval(example[0]))  # one is enough
    except:
        print([])
        examples.append([])

with open("examples.pickle", "wb") as file:
    pickle.dump(examples, file)

  0%|          | 1/221 [00:06<23:47,  6.49s/it]

["Die meisten von ihnen haben zuvor noch nie einen Computer gesehen", "Die meisten von ihnen haben zuvor noch nie einen Computer bedient"]


  1%|          | 2/221 [00:14<27:33,  7.55s/it]

["Die Kern-Konstruktionselemente sind 'Verglichene_Entitäten' und 'Dimension'", "Die Skala wird durch die 'Dimension' angegeben", "Die Konstruktion zeigt die Gleichheit zweier Entitäten an, die hinsichtlich eines bestimmten Wertes eines spezifischen Attributs auf einer Skala miteinander verglichen werden", "Die Konstruktion gehört zu den Vergleichskonstruktionen", "Die Konstruktion zeigt die Gleichheit zweier Entitäten an, die hinsichtlich eines bestimmten Wertes eines spezifischen Attributs auf einer Skala miteinander verglichen werden"]


  1%|▏         | 3/221 [00:16<17:56,  4.94s/it]

...

["Wo Xist, ist Y", "Wo Xist, ist Y", "Wo Xist, ist Y", "Wo Xist, ist Y", "Wo Xist, ist Y"]


100%|█████████▉| 220/221 [22:12<00:06,  6.65s/it]

["Die Blätter sind so grün wie die Wände", "Das Auto fährt so schnell wie das Motorrad", "Das Haus ist so alt wie das Rathaus", "Das Buch ist so schwer wie das Tablet", "Das Essen schmeckt so gut wie das Gebäck"]


100%|██████████| 221/221 [22:18<00:00,  6.06s/it]

["Die Größe von X ist größer als die Größe von Y", "Die Größe von X ist kleiner als die Größe von Y", "Die Größe von X ist gleich der Größe von Y", "Die Größe von X ist größer als die Größe von Y", "Die Größe von X ist kleiner als die Größe von Y"]
